**IA & Big Data**

Prof. Miguel Bozer da Silva - miguel.bozer@senaisp.edu.br

---

In [ ]:
# Tratamento dos dados
import numpy as np
import pandas as pd

# Modelos de Machine Learning
from sklearn.model_selection import train_test_split
from sklearn.discriminant_analysis import QuadraticDiscriminantAnalysis, LinearDiscriminantAnalysis
from sklearn.naive_bayes import GaussianNB
from sklearn.svm import SVC

from sklearn.preprocessing import StandardScaler, MinMaxScaler

# Métricas de avaliação
from sklearn.metrics import f1_score, roc_auc_score, roc_curve, ConfusionMatrixDisplay, RocCurveDisplay, confusion_matrix, classification_report

# Plot dos gráficos
import matplotlib.pyplot as plt
from matplotlib import colors
import seaborn as sns

# Recebendo os dados:
from googledrivedownloader import download_file_from_google_drive as gdd

Entrada do projeto:

- buying:   vhigh, high, med, low. (Preço de Compra: muito alto - 4, alto - 3, médio - 2  e baixo - 1)
- maint:    vhigh, high, med, low. (Preço da Manutenção: muito alto - 4, alto - 3, médio - 2  e baixo - 1)
- doors:    2, 3, 4, 5more. (nº de portas: 2, 3, 4 e 5 ou mais)
- persons:  2, 4, more. (nº de pessoas: 2, 4 e mais)
- lug_boot: small, med, big. (tamanho do porta malas: pequeno, médio e grande)
- safety:   low, med, high. (segurança estimada do carro: baixo, médio e alto)

Saída do projeto:
- category: unacceptable, acceptable, good, very good  (Categoria/Nível de avaliação: inaceitável, aceitável, bom e muito bom)

# Preparando os dados

## Tarefa #1: Recebendo os dados

In [ ]:
# Recebendo os dados:
data_google_id = '1QbuZTpEKKLzr5gUXK0I9bnOfv0qGg3Lb'
gdd(file_id=data_google_id,
    dest_path = './dados.csv', # Faz o download dos dados e salva o mesmo num arquivo nomeado data.csv
    showsize = True)

# Armazenandos os dados em um DataFrame
# para receber os dados o sep teve como argumento o valor ','. Isso ocorreu devido a
dados = pd.read_csv("dados.csv", sep = ',')

## Tarefa #2: Analise Exploratória dos Dados

In [ ]:
dados.head()

A coluna `Unnamed: 0` tem apenas os números dos index do nosso conjunto de dados. Deveremos remover essa coluna.

In [ ]:
dados.shape

In [ ]:
dados.info()

Temos colunas do tipo object. Além disso, segunda as informações obtidas do conjunto de dados temos diversas colunas que são categóricas mas que já estão convertidas em valores numéricos. Entretanto teremos que tratar as colunas que estão com valores object.

In [ ]:
dados.isnull().sum()

Temos valores nulos no nosso conjunto de dados. Logo DEVEMOS fazer o tratamento disso removendo os valores nulos

In [ ]:
dados.describe()

Todas as colunas possuem valores na mesma ordem de grandeza. Logo não haverá necessidade de transformar as escalas dos valores de cada coluna.

## Tarefa#3: Tratamento dos dados

Removendo a coluna sem dados informativos:

In [ ]:
dados.drop(columns=['Unnamed: 0'], inplace = True)

Removendo os valores nulos:

In [ ]:
dados.dropna(inplace=True)
dados.isnull().sum()

Avaliando os valores únicos em cada coluna:

In [ ]:
colunas = dados.columns
for col in colunas:
  print('='*60)
  print(col)
  print(dados[col].unique())
  print('='*60)

Vamos aplicar o One Hot encoding nas colunas `doors` e `persons`:

In [ ]:
one_hot_doors = pd.get_dummies(dados['doors'], prefix='doors')
one_hot_doors.head()

In [ ]:
one_hot_persons = pd.get_dummies(dados['persons'], prefix='persons')
one_hot_persons.head()

In [ ]:
dados = pd.concat([dados, one_hot_doors, one_hot_persons], axis =1)
dados.head()

Agora vamos fazer um drop nas colunas originais com valores do tipo object:

In [ ]:
dados.drop(columns=['doors','persons'], inplace = True)

In [ ]:
dados.head()

# Aplicando IA aos nossos dados

## Tarefa #1: Dividindo o conjunto de dados (Entrada/Saída e Treinamento/Teste)

In [ ]:
# Separando os dados de entrada e de saída
X = dados.drop(columns=['category'])
y = dados["category"] #Rótulos/Labels

# Dividindo os dados no conjunto de teste e treinamento
X_train, X_test, y_train, y_test = train_test_split(X,
                                                    y,
                                                    test_size=0.2,
                                                    random_state=25)

## Tarefa #2: Treinando os modelos

In [ ]:
modelos = {}
y_pred = {}
for nome_modelo, modelo in modelos.items():
  # Treinar o modelo:
  modelo.fit(X_train, y_train)
  y_pred[nome_modelo] = modelo.predict(X_test)

## Tarefa #3: Métricas de desempenho do modelo

Obtendo as matrizes de confusão:

In [ ]:
figure = plt.figure(figsize=(25,10))
for i, (nome_modelo, y_p) in enumerate(y_pred.items()):
  cm = confusion_matrix(y_true=y_test,
                        y_pred=y_p,
                        labels=)
  disp = ConfusionMatrixDisplay(confusion_matrix=cm,
                                display_labels=)
  ax = plt.subplot(2,2,i+1)
  disp.plot(ax=ax)
  ax.set_title(nome_modelo)

Obtendo as métricas:

In [ ]:
for nome_modelo, y_p in y_pred.items():
  print(f"Métricas do modelo: {nome_modelo}")
  print(classification_report(y_true=y_test,
                              y_pred=y_p))
  print('-'*53)

Podemos perceber que o modelo do SVC foi o que obteve o melhor desempenho nesse cenário. Além disso, o QDA encontrou problemas devido as ferramentas matemáticas que ele usa foram problematicas para os atributos do problema.